Firstly, add $__init__.py$ from utilities dir (to do it tap < in the left)  

Secondly, rename $__init.py__$ to $ utilities.py $ 

To install msprime we need to get some dev tools

In [0]:
!apt-get install python-dev libgsl0-dev
!python3 -m pip install msprime

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libgsl-dev' instead of 'libgsl0-dev'
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libgsl23 libgslcblas0
Suggested packages:
  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html
The following NEW packages will be installed:
  libgsl-dev libgsl23 libgslcblas0
0 upgraded, 3 newly installed, 0 to remove and 7 not upgraded.
Need to get 1,926 kB of archives.
After this operation, 9,474 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgslcblas0 amd64 2.4+dfsg-6 [79.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl23 amd64 2.4+dfsg-6 [823 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe a

In [0]:
import utilities

Create data generator 

In [0]:
events = utilities.generate_demographic_events()

In [0]:
Ne = 1
rho =  1.6*10e-9*Ne
mu  = 1.25*10e-8*Ne
num_repl = int(1e5) # 5
l = int(3e3)

dg = utilities.DataGenerator(recombination_rate=rho,
                             mutation_rate=mu,
                             demographic_events=events,
                             num_replicates=num_repl, lengt=l)
data = dg.run_simulation()

Create datasets

In [0]:
number_train_examples = int(num_repl*.9)

trX, trY = [], []
for i in range(number_train_examples):
  example = next(dg)
  trX.append(example[0])
  trY.append(example[1])

teX, teY = [], []
for example in dg:
  teX.append(example[0])
  teY.append(example[1])


del dg

In [0]:
import torch

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        assert x.shape[0] == y.shape[0] # assuming shape[0] = dataset size
        self.x = x
        self.y = y


    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [0]:
import numpy as np
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# device = 'cpu'

input = torch.from_numpy(np.array(trX, dtype=np.float_)) #.to(device)
target = torch.from_numpy(np.array(trY)) # .to(device)
test_input = torch.from_numpy(np.array(teX, dtype=np.float_))# .to(device)
test_target = torch.from_numpy(np.array(teY))# .to(device)

del trX, trY, teX, teY

traindata = MyDataset(input, target)

cuda


Specify NN

In [0]:
# https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

N = 51

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step

loss_per_step = []
def main():
    # set random seed to 0
    np.random.seed(0)
    torch.manual_seed(0)
    # load data and make training set
    train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, 
                                              shuffle=True)

    # build the model
    seq = Sequence()
    seq = seq.to(device)
    seq.double()
    criterion = nn.MSELoss()
    criterion = criterion.to(device)
    # 
    optimizer = optim.SGD(seq.parameters(), lr=.001)
    #begin to train
    
    train_step = make_train_step(seq, criterion, optimizer)
    total_step = len(train_loader)
    
    for i in range(15):
        print('STEP: ', i)
        total_loss = 0
        for i, (x_batch, y_batch) in enumerate(train_loader):
          loss = train_step(x_batch.to(device), y_batch.to(device))
          total_loss += loss
          if i % 5 == 0: print(f'BATCH: {i+1}/{total_step} LOSS: {loss}')
        loss_per_step.append(total_loss)

In [0]:
# %%script false
main()
loss_per_step

STEP:  0
BATCH: 1/1407 LOSS: 5191889.266543173
BATCH: 6/1407 LOSS: 4645469.583396519
BATCH: 11/1407 LOSS: 5293222.802241087
BATCH: 16/1407 LOSS: 4065448.650027007
BATCH: 21/1407 LOSS: 2107837.633462674
BATCH: 26/1407 LOSS: 3427755.312646055
BATCH: 31/1407 LOSS: 1493515.819444495
BATCH: 36/1407 LOSS: 2148274.099311213
BATCH: 41/1407 LOSS: 1276548.5994584775
BATCH: 46/1407 LOSS: 2958069.2189081684
BATCH: 51/1407 LOSS: 3960645.8394038845
BATCH: 56/1407 LOSS: 2638955.72074782
BATCH: 61/1407 LOSS: 2497273.5393991075
BATCH: 66/1407 LOSS: 3682395.924624732
BATCH: 71/1407 LOSS: 2253107.2372786095
BATCH: 76/1407 LOSS: 1740091.69749368
BATCH: 81/1407 LOSS: 2135142.4117426467
BATCH: 86/1407 LOSS: 2372261.8022592436
BATCH: 91/1407 LOSS: 1756205.0499965185
BATCH: 96/1407 LOSS: 2216065.623397198
BATCH: 101/1407 LOSS: 2165881.144110333
BATCH: 106/1407 LOSS: 1612537.7478806279
BATCH: 111/1407 LOSS: 2445657.257538476
BATCH: 116/1407 LOSS: 2557552.0849812333
BATCH: 121/1407 LOSS: 2957930.963734914
BATCH

In [0]:
# from https://github.com/pytorch/examples/blob/master/time_sequence_prediction/train.py

# It's work, but only for little dataset - we need to specify mini-batches 
# to make it work;

%%script false

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

N = 51

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

def main():
    # set random seed to 0
    np.random.seed(0)
    torch.manual_seed(0)
    # load data and make training set

    #data = torch.load('traindata.pt')
    input = torch.from_numpy(np.array(trX, dtype=np.float_))
    target = torch.from_numpy(np.array(trY))
    test_input = torch.from_numpy(np.array(teX, dtype=np.float_))
    test_target = torch.from_numpy(np.array(teY))
    
    # build the model
    seq = Sequence()
    seq.double()
    criterion = nn.MSELoss()
    # use LBFGS as optimizer since we can load the whole data to train
    optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
    #begin to train
    for i in range(15):
        print('STEP: ', i)
        def closure():
            optimizer.zero_grad()
            out = seq(input)
            loss = criterion(out, target)
            print('loss:', loss.item())
            loss.backward()
            return loss
        optimizer.step(closure)
        # begin to predict, no need to track gradient here
        with torch.no_grad():
            future = 1000
            pred = seq(test_input, future=future)
            loss = criterion(pred[:, :-future], test_target)
            print('test loss:', loss.item())
            y = pred.detach().numpy()

https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

https://towardsdatascience.com/lstm-for-time-series-prediction-de8aeb26f2ca

http://chandlerzuo.github.io/blog/2017/11/darnn